In [2]:
import numpy as np 
import pandas as pd
import copy
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision
from torchvision import models,transforms
import cv2
from tqdm import tqdm
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
import albumentations
import joblib

In [3]:
train_img_path = "../input/age detection of actors/train/Train"
train_df = pd.read_csv("../input/age detection of actors/train/train.csv")

test_img_path = "../input/age detection of actors/test/Test"
test_df = pd.read_csv("../input/age detection of actors/test/test.csv")

sub_df = pd.read_csv("../input/age detection of actors/sample_submission.csv")

In [4]:
train_df.Class.unique()


array(['MIDDLE', 'YOUNG', 'OLD'], dtype=object)

In [5]:
train_df["kfold"] = -1
kf = StratifiedKFold(n_splits = 5,shuffle=False,random_state=42)
train_df.sample(frac=1).reset_index(drop=True)

for fold_,(train_,val_) in enumerate(kf.split(X=train_df,y=train_df.Class.values)):
    train_df.loc[val_,"kfold"] = fold_
    print(len(train_),len(val_))

15924 3982
15925 3981
15925 3981
15925 3981
15925 3981


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [6]:
train_df["kfold"].value_counts()

0    3982
4    3981
3    3981
2    3981
1    3981
Name: kfold, dtype: int64

In [7]:
for i in range(train_df.shape[0]):
    if train_df.loc[i,"Class"]=='YOUNG':
        train_df.loc[i,"Class"] = 0
    if train_df.loc[i,"Class"]=="MIDDLE":
        train_df.loc[i,"Class"] = 1
    if train_df.loc[i,"Class"]=="OLD":
        train_df.loc[i,"Class"] = 2

In [8]:
train_df.Class.unique()

array([1, 0, 2], dtype=object)

In [9]:
class ActorDataset:
    def __init__(self,image_list,target,aug=None,new_dim=None):
        self.image_list = image_list
        self.aug = aug
        self.target=target
        self.new_dim = new_dim
    
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self,idx):
        
        image = self.image_list[idx]
        image = Image.open(image)
        if self.new_dim is not None:
            image = image.resize(self.new_dim)
        if self.aug is not None:
            image = self.aug(image=np.array(image))["image"]

        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        return {
            "image":torch.tensor(image,dtype=torch.float),
            "targets":torch.tensor(self.target[idx],dtype = torch.long)
        }
        

In [10]:
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 15
MODEL = "vgg"
NUM_CLASSES = 3
DEVICE ="cuda"

In [11]:
def set_parameters_grad(model,feature_extract = False):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False

In [17]:
def initialize_model(model,num_class,feature_extract=True,pretrained=True):
    
    model_ft = None
    input_size = 0
    
    if model == "resnet":
        model_ft = models.resnet50(pretrained = pretrained)
        set_parameters_grad(model_ft,feature_extract)
        num_ftrs = model.fc.in_features
        model_ft.fc = nn.linear(num_ftrs,num_class)
        input_size = (224,224)
    
    elif model == "vgg":
        model_ft = models.vgg11_bn(pretrained=pretrained)
        set_parameters_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,3)
        input_size = (224,224)
    
    elif model == "alexnet":
        model_ft = models.alexnet(pretrained=pretrained)
        set_parameters_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = (224,224)
    
    elif model == "densenet":
        self.model = models.densenet121(pretrained = pretrained)
        set_parameters_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_class)
        input_size = (224,224)
    
    elif model == "squeezenet":
        model_ft = models.squeezenet1_0(pretrained=pretrained)
        set_parameters_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_class
        input_size = (224,224)
    
    else:
        print("invalid model name")
    
    return model_ft,input_size

In [23]:
def train_model(model,dataloader,criterion,optimizer,device=DEVICE,epochs=5):
    val_acc_history = []
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(epochs):
        print(f"EPOCH:{epoch}/{epochs-1}")
        print("-"*20)
        
        for phase in ["train","val"]:
            if phase == "train":
                model.train()
                bs = TRAIN_BATCH_SIZE
            else:
                model.val()
                bs = VAL_BATCH_SIZE
        
            running_loss = 0.0
            running_corrects = 0

            for i,batch in enumerate(dataloader[phase]):
                inputs = batch["image"]
                labels = batch["targets"]
                    
                if phase == "train":
                    optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                _, preds = torch.max(outputs, 1)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

                running_corrects += torch.sum(preds == labels.data)
                accuracy = running_corrects.double() / len(dataloader[phase].dataset)

                print(f"{phase} LOSS:{loss} , {phase} ACC:{accuracy}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        
    
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model,val_acc_history
            
            

In [21]:
def run(fold):
    train = train_df[train_df.kfold.values!=fold]
    val = train_df[train_df.kfold.values==fold]

    train_list = [os.path.join(train_img_path,train.ID.values[i]) for i in range(len(train.ID.values.tolist()))]
    val_list = [os.path.join(train_img_path,val.ID.values[i]) for i in range(len(val.ID.values.tolist()))]
    
    train_target = train.Class.values.tolist()
    val_target = val.Class.values.tolist()

    model_ft,input_size = initialize_model(MODEL,num_class = NUM_CLASSES,feature_extract=False,pretrained=True)
    
    train_aug = albumentations.Compose([
        albumentations.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]),
        albumentations.RGBShift(p=0.3),
        albumentations.RandomBrightnessContrast(p=0.3)
        
    ])
    
    train_dataset = ActorDataset(train_list,target=train_target,aug= train_aug,new_dim=input_size)

    val_aug = albumentations.Compose([
        albumentations.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
    ])
    
    val_dataset = ActorDataset(val_list,target=val_target,aug= train_aug,new_dim=input_size)

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = TRAIN_BATCH_SIZE,
        shuffle = False,
        num_workers = 4

    )

    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size = VAL_BATCH_SIZE,
        shuffle = False,
        num_workers = 4

    )

    dataloader = {
        "train":train_dataloader,
        "val":val_dataloader
    }
    optimizer = torch.optim.SGD(model_ft.parameters(),lr = 3e-4, momentum = 0.9)
    lossfn = nn.CrossEntropyLoss()
    model,val_accuracy = train_model(model_ft,dataloader,criterion = lossfn,optimizer=optimizer,epochs=5)
    model_path = os.path.join("../kaggle/working/fold_models",f"model_fold_{fold}.bin")
    torch.save(model.state_dict(), model_path)



In [ ]:
os.mkdir("/kaggle/working/fold_models")

In [ ]:
if __name__=="__main__":
    run(0)
    run(1)
    run(2)
    run(3)
    run(4)